In [1]:
import tushare as ts
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pro = ts.pro_api('token')
mf = ["161028.OF","163406.OF","340007.OF","001740.OF","161725.OF","257070.OF","007300.OF","001717.OF","001594.OF","006585.OF","008086.OF","010319.OF","000118.OF"]

In [2]:
import urllib
import json
url="http://192.168.1.150:8112/manage/xuangu/all_stock"
    
res = urllib.request.urlopen(url)
d = json.loads(res.read().decode('utf-8'))


In [3]:
#所有股票
stocks = pro.stock_basic()
stocks.head()

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,国华网安,深圳,软件服务,主板,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427


In [4]:
#基金持仓股票统计
df = pro.fund_portfolio(start_date="20200930")

jdfa = '000725.SZ'
gzmt = '600519.SH'
sy = df.loc[df['symbol'] == jdfa]
c = df['symbol'].value_counts()    
df = stocks.loc[ lambda df : df['ts_code'].isin(list(c.index))].copy()
df.loc[:,'count'] = df.apply(lambda x : c[x['ts_code']] ,axis=1)
df = df.loc[df['count'] > 2]
df = df.sort_values('count', ascending = False)
df.head()

,ts_code,symbol,name,area,industry,market,list_date,count
346,000858.SZ,000858,五粮液,四川,白酒,主板,19980427,112
2807,600519.SH,600519,贵州茅台,贵州,白酒,主板,20010827,102
862,002415.SZ,002415,海康威视,浙江,电器仪表,中小板,20100528,87
3206,601012.SH,601012,隆基股份,陕西,电气设备,主板,20120411,84
69,000333.SZ,000333,美的集团,广东,家用电器,主板,20130918,80


In [5]:
df = pro.fund_nav(ts_code="340007.OF")
df.head()

,ts_code,ann_date,end_date,unit_nav,accum_nav,accum_div,net_asset,total_netasset,adj_nav,update_flag
0,340007.OF,20210420,20210419,5.546,5.736,None,NaN,NaN,6.319671,0
1,340007.OF,20210417,20210416,5.358,5.548,None,NaN,NaN,6.105445,0
2,340007.OF,20210416,20210415,5.372,5.562,None,NaN,NaN,6.121398,0
3,340007.OF,20210415,20210414,5.409,5.599,None,NaN,NaN,6.163559,0
4,340007.OF,20210414,20210413,5.308,5.498,None,NaN,NaN,6.048470,0


In [6]:
df_share = pro.fund_share(ts_code = "010160.OF",end_date="20201208")
df_share.head()

,ts_code,trade_date,fd_share,fund_type,market
0,010160.OF,20200930,806.4429,None,O


In [7]:

df['ff'] = df['adj_nav'].diff(-1)/ df['adj_nav'].shift(-1)


rows = df[df['end_date'] == '20201105'].index.values
cc = round(((df.iloc[0]['adj_nav'] - df.iloc[rows]['adj_nav']) / df.iloc[rows]['adj_nav'])*100,2)
# df.head(30)
rows[0]

111

In [8]:
cal = pro.trade_cal(start_date="20201101").loc[:,['cal_date','is_open']]


In [9]:
import datetime
import pendulum
date = datetime.datetime.strptime('20200229', '%Y%m%d')
# year = int(date.tm_year) - 2
# date.tm_year = year
# time.strftime('%Y%m%d',date)
today = datetime.datetime.today()
y = pendulum.datetime(today.year, today.month,today.day).subtract(days=1)
c = pendulum.datetime(date.year, date.month, date.day).subtract(months=1)
y

DateTime(2021, 4, 19, 0, 0, 0, tzinfo=Timezone('UTC'))

In [11]:
df = pro.fund_manager(limit="100", name="董承非")
df = df.loc[df['end_date'].isnull()]
g = df.groupby('name')

arr = []
for ts,item in g:
    
#     item.reset_index(inplace = True)
#     obj = dict(
#         name=ts,
#         edu = item.loc[0,'edu'],
#         birth_year = item.loc[0,'birth_year'],
#         gender = item.loc[0,'gender'],
#         nationality = item.loc[0,'nationality'],
#         resume = item.loc[0,'resume'],
#         funds = [ { "ts_code":i.ts_code, "begin_date":i.begin_date } for i in item.itertuples() ]
#     )    
    print(item)
    # arr.append(obj)
arr



     ts_code  ann_date name gender birth_year edu nationality begin_date  \
0  001511.OF  20150702  董承非      M       1977  硕士          中国   20150701   
1  163402.SZ  20131026  董承非      M       1977  硕士          中国   20131028   

  end_date                                             resume  
0     None  理学硕士。2003年9月加入兴业全球基金管理有限公司，历任研究策划部行业研究员、兴全趋势投资...  
1     None  理学硕士。2003年9月加入兴业全球基金管理有限公司，历任研究策划部行业研究员、兴全趋势投资...  


[]